<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/WebData_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Web Data Url Validation and data extraction**

This Code goes through each of the urls populated in the queue. <br/>
Fetches the Url and validates if its a valid url and then extracts the contents

Install the pre-requesites <br/>
https://pypi.org/project/azure-storage-queue/ <br/>
https://pypi.org/project/azure-data-tables/


In [10]:

# try:
#   % azure-storage-queue
# except Exception:
#   pass


!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables

ERROR: Could not find a version that satisfies the requirement azure-storages (from versions: none)
ERROR: No matching distribution found for azure-storages


**Initialize Variables**

In [11]:
webPageReadTimeout = 10
baseUrl= "https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html" # For testing
queue_connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
queue_name = "queue-crawledarchiveurls"

Intialize the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md


In [12]:

from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid

queue_service = QueueService(connection_string=queue_connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode

 

In [13]:
def GetQueueCount(queueName):
  properties = queue_service.get_queue_metadata(queueName)
  queueCount = properties.approximate_message_count
  print("Message count: " + str(queueCount))
  return queueCount

Method to Check if the url exists and returns True if valid

In [14]:
import requests

def check_url_exists(url: str):
  try:
    response = requests.get(url)
    if response.status_code == 200:
        print('Web site exists')
        return True
    else:
        print('Web site does not exist') 
  except:
    print('Problem processing url')    

  return False

In [15]:
#Testing
check_url_exists("https://www.google.com")

Web site exists


True

Below function takes a url converts into a Soap object and returns for further processing. Such as taking the url from href object


In [16]:
#Fetch the data from web pages via Beautiful soup

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      print("getSoupObj",url)
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)


In [17]:
#Method to get text from link
def getTextFromLink(link): 
  page = urlopen(link)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  pagetext =soup.get_text()
  return pagetext
  

In [18]:
#Fetch the data from web pages via Beautiful soup

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      print("getSoupObj",url)
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)


In [ ]:
#Testing
from bs4 import BeautifulSoup
from urllib.request import urlopen
a = getSoupObj(baseUrl)
b = a.findAll('a')
print(len(b))

for link in a.findAll('a'):
  if link.get('href') != None:
    print(link.get('href'))


In [24]:
def GetDataFortheMessage(message):
  # arrHeader = ['Url','MessageInJsonFormat','IsValidUrl','HTMLData','TextData']
  arr= [0] * len(arrHeader)

  arr[1]=json.dumps(message.content, default=vars) #Update Array
  print("Printing message object from queue: " + json.dumps(message.content, default=vars))
  # print("Printing message object from queue:" + message)
  url = message #Get url from complex object
  arr[0] =url #Update Array
  if check_url_exists(url) == False:
    arr[2] = False
    print("Invalid Url - Deleting message: " + message.content)
    queue_service.delete_message(queue_name, message.id, message.pop_receipt) #Delete if Invalid Url
  else:
    arr[2] = True
    soupObj= getSoupObj(url)
    arr[3]= requests.get(url).text #Get HTML
    arr[4] = soupObj.get_text() #Get Text
    queue_service.delete_message(queue_name, message.id, message.pop_receipt) #Delete after fetching
    print('Url exists') 
  return arr

**Process each Urls in the Queue to check if it is valid and delete if invalid.** <br/>

https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md <br/>
https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/storage/azure-storage-queue/samples/queue_samples_message.py


In [25]:
import json
arrHeader = ['Url','MessageInJsonFormat','IsValidUrl','HTMLData','TextData']
details= []
queue_Count= GetQueueCount(queue_name)
print("Azure Queue Count" +str(queue_Count))
if queue_Count >0:
  messages = queue_service.get_messages(queue_name)
  print(messages)
  if len(messages) > 0:
    for message in messages:
        details.append(GetDataFortheMessage(message))
  else:
    print('No messages found') 



Message count: 0
Azure Queue Count0


**Use Pandas to Create a Data frame to convert it as CSV**

In [26]:

import pandas as pd
df = pd.DataFrame(details, columns= arrHeader)
#Check top few records
df

,Url,MessageInJsonFormat,IsValidUrl,HTMLData,TextData


Mount google drive folder to store the csv file

In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

from datetime import datetime
now = datetime.now() # current date and time

csvWritePath = '/content/gdrive/My Drive/DissertationFiles/germanarchive.csv'+now.strftime("%m_%d_%Y__%H_%M_%S")+'.csv'


Mounted at /drive


In [28]:
#Write Dataframe to CSV file
df.to_csv(csvWritePath, index=False)